In [20]:
import os

import numpy as np
import pandas as pd
import cv2
from PIL import Image, ImageSequence
import PIL.ImageOps
from sklearn.model_selection import train_test_split
from sklearn import datasets
#import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

#import ray 

from bs4 import BeautifulSoup as BSHTML
import urllib
import requests

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
from sklearn.manifold import MDS
from sklearn.manifold import Isomap 
from sklearn.manifold import LocallyLinearEmbedding as LLE
from sklearn.manifold import SpectralEmbedding as SE
from sklearn.random_projection import GaussianRandomProjection as GRP
#import umap
import umap.umap_ as umap

from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn import preprocessing

In [ ]:
def createFolders(path): 
    files = os.listdir(path)
    for f in files:
        print(f)
        if os.path.isdir(path) and not f.startswith('.') and not f.startswith('_'):
            path = os.getcwd() + '/' + f 
            if not os.path.exists(path + '/thumbnails'):
                os.mkdir(path + '/thumbnails')
            if not os.path.exists(path + '/thumbnails200'):
                os.mkdir(path + '/thumbnails200')
            if not os.path.exists(path + '/data'):
                os.mkdir(path + '/data')
            if not os.path.exists(path + '/data200'):
                os.mkdir(path + '/data200')
            if not os.path.exists(path + '/figures'):
                os.mkdir(path + '/figures')
            if not os.path.exists(path + '/figures200'):
                os.mkdir(path + '/figures200')
  

In [ ]:
def rename_images(folder, splitSymbol):
    files = os.listdir(folder)

    for f in files:
        print(f)
        if f.startswith('.'):
            continue
        new_f = f.split(splitSymbol)[-1] #001.jpg
        if not os.path.exists(folder+'/'+ new_f):
            os.mkdir(folder +'/'+ new_f)              

        for img in os.listdir(folder+'/'+new_f):
            print(img)
            new_img = f + "_" + img.split(splitSymbol)[-1]
            print(new_img)
            os.replace(folder+'/'+ new_f + "/" + img , folder + '/'+new_f + "/" + new_img)
        

In [61]:
def resize_img(file, destfolder, dim=[50,50]):
    print(file)
    img = cv2.imread(file, cv2.IMREAD_UNCHANGED)
    try:
        print(img.shape)
    except:
        return
    
    for i in [0,1]:
        if img.shape[i]<dim[i]:
            dim[i] = img.shape[i]
        
    dim = tuple(dim)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite(file.split('/')[0] + '/'+destfolder+'/'  + file.split('/')[-1], resized)

def resize_gif(dir_path, destfolder, dim = [50,50]):
    files = os.listdir(dir_path)
    
    for f in files:
        if f.endswith('.gif'):
            print(f)
            try:
                im = Image.open(dir_path+'/'+f)
            except: 
                return
            im = Image.open(dir_path+'/'+f)

            # Get sequence iterator
            frames = ImageSequence.Iterator(im)

            def thumbnails(frames):
                for frame in frames:
                    thumbnail = frame.copy()
                    thumbnail.thumbnail(dim, Image.ANTIALIAS)
                    yield thumbnail

            frames = thumbnails(frames)

            # Save output
            om = next(frames) # Handle first frame separately
            om.info = im.info # Copy sequence info
            newpath = dir_path.split('/')[0]+'/'+destfolder+'/'+f
            om.save(newpath, save_all=True, append_images=list(frames))

def resize_all(path, destfolder, dim=[50,50]):
    files = os.listdir(path)
    for f in files: 
        print(f)
        if os.path.isdir(path + "/" + f) and not f.startswith('.') and not f.startswith('_') and not f == destfolder:
            subfiles = os.listdir(path + "/" + f)
            for ff in subfiles:
                print(ff)
                if ff.endswith('.png') or ff.endswith('.jpg'):
                    resize_img(path  +'/'+ f+ "/" + ff,destfolder, dim=dim)
                elif ff.endswith('.gif'):
                    resize_gif(path  +'/'+ f,destfolder, dim=dim)                  

        elif f.endswith('.png') or f.endswith('.jpg') or f.endswith('.gif'):
            if f.endswith('.jpg'):
                new_name = path  + f.split('.')[0] + '.png'
                os.rename(path  + f, new_name)
            
                resize_img(new_name, destfolder, dim=dim)
            elif f.endswith('.gif'):
                resize_gif(path  +'/'+ f,destfolder, dim=dim)   
            else:
                resize_img(path +'/'+ f,destfolder, dim=dim)             
        else:
            print('No resizeing')
        print('------')
    
def save_images(data, label, name='MNIST'):
    #create folder for each dataset
    save_folder = os.getcwd() + '/' + name +'/'
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
        
    save_folder +='thumbnails/'
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    
    print(save_folder)
    print(data.shape)
    
    #if data is in image version 
    if len(data.shape) > 2:
        pass
    #if data is flat
    if len(data.shape) == 2:
        for i in range(data.shape[0]):

            img = np.array(data[i].reshape(64, 64)).astype(np.uint8)
            
            img = Image.fromarray(img)
            
            img = PIL.ImageOps.invert(img)
            
            img.save(save_folder + '{}_{}.png'.format(label[i],i))
        
def resize_csv(csv): 
    x_test = pd.read_csv(csv)
    name = x_test['name'].astype(str)
    y_test = x_test['class']+'_' +name
    print(y_test)

    x_test = x_test.iloc[:,2:]
    x_test = np.array(x_test)
    print(x_test.shape)

    image_index = 1
    plt.imshow(x_test[image_index].reshape(64,64), cmap='Reds')
    plt.axis('off') 
    save_images(x_test, y_test, name='stf')
     

In [74]:
def flatten_images(path, number=50):
    if not os.path.exists(path):
        return 1
    
    
    if os.path.exists(path.split('/')[0]+'/flat_img_{}.csv'.format(number)):
        print('Skipping....')
        return 
    images = []
    names = []
    labels = []
    for i,ff in enumerate(os.listdir(path)):
        if ff.startswith('.'):
            print('not an image: ' + ff)
            continue
        if i <2:
            print(ff.split('.')[0])
        try:
            img = Image.open(path +'/'+ ff)
        except: 
            print('cannot open: ' + ff)
            continue
            
        img = Image.open(path +'/'+ ff)
        if img is None:
            print('!!!! '+ ff)
            continue
#         if len(img.shape) > 2 and img.shape[2] == 4:
#             #convert the image from RGBA2RGB
#             try:
#                 img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
#             except:
#                 continue

        img = pd.DataFrame(np.array(img).flatten()).T
        print('image: ' +ff)
        #print(img)
        images.append(img)
        names.append(ff.split('.')[0])
        labels.append(ff.split('_') [0])
#         print(names)
#         print(labels)

    if len(images)>0:
        imgs = pd.concat(images, axis=0)
        imgs['filename'] = names
        imgs['labels'] = labels#['.'.join(n.split('.')[:-1]) for n in names]
        #print(imgs.head())
        imgs = imgs.dropna(axis=1)

        
        imgs.to_csv(path.split('/')[0]+'/'+path.split('/')[1]+'/flat_img_{}.csv'.format(number),index=False)
        print(imgs.head(10))
        print(imgs.shape)
    print('----Finished: {}----'.format(path))

    return 0

files = ['stf']
for number in [50,200]:
    for f in files:
        if os.path.isdir(f) and not f.startswith('.') and not f.startswith('_') and not f.startswith('MNIST') and not f.startswith('textures'):
            print(f)
            if number == 50:
                path = f+'/thumbnails{}/'.format('')
            else:
                path = f+'/thumbnails{}/'.format(number)
            if not os.path.exists(path):
                print('x')
                continue
            else:
                flatten_images(path, number)


In [67]:
files = ['stf']
classes = [15,10,10,10,10,10,10,10]
number = 50


def reformat(f, number, c=10):
    print('Started: {}'.format(f))
    path = f+'/flat_img_{}.csv'.format(number)
    
#     if os.path.exists(f+'/flat_img_{}_{}classes.csv'.format(number, c)):
#         print('already done')
#         return
    
    try:
        df = pd.read_csv(path)
    except e:
        print('didnt work')
        print(e)
        return

    print('Before: {}'.format(df.shape))
    df_agg = df.groupby(['labels']).count()['filename'].reset_index().sort_values(['filename'], ascending=False)
    ll = df_agg.reset_index().loc[:c, 'labels']
    print('{} \n ---- \n {}'.format(f,df_agg.head(c)))
    df = df.loc[df['labels'].isin(ll)]
    df.to_csv(f+'/flat_img_{}_{}classes.csv'.format(number, c), index=False)
    print('After: {}'.format(df.shape))
    print('Finished: {}'.format(f))
    return 1


for number in [50]:
    for f in files:
        reformat(f, number)
            

In [18]:
#@ray.remote
def apply_dr(datafolder, figurefolder, dr_name, file_name, perplexity = None, n_neighbors=None, min_dist=None, dataset_name = 'MNIST/', max_samples = 5000):
    print('---------Starting: {}-----------'.format(file_name))
    df = pd.read_csv(file_name)
    
    print(file_name)
#     if os.path.exists(dataset_name + '/' + datafolder+'/' + file_name + '.csv'):
#         print('File was already calculated. Skipping ....')
#         return

    y = df['labels']
    X = df.iloc[:,:-2]
    
    if df.shape[0]>max_samples:
        X_train, features, y_train, labels = train_test_split(X, y, test_size=max_samples, random_state=42, stratify=y)
    else: 
        X_train, features, y_train, labels = train_test_split(X, y, test_size=0.7, random_state=42, stratify=y)
    
    
    file_name = "{dr_name}{perp}{neigh}{mindist}".format(dr_name = dr_name, 
                                         perp = "_p" + str(perplexity) if perplexity is not None else "",
                                         neigh = "_n" + str(n_neighbors) if n_neighbors is not None else "",
                                         mindist = "_d" + str(min_dist) if min_dist is not None else "")
    
    
    ## check if files exist
    print(file_name)
    if os.path.exists(dataset_name + '/' + datafolder + '/' + file_name + '.csv'):
        print('File was already calculated. Skipping ....')
        return
    ########
    
    ## apply dr
    if dr_name=='PCA':
        dr = PCA(n_components = 2)
        
    elif dr_name=='TSNE':
        dr = TSNE(n_components = 2, perplexity = perplexity, verbose = 1)
        
    elif dr_name=='ISM':
        dr = Isomap(n_components = 2, n_neighbors = n_neighbors)
        
    elif dr_name=='LLE':
        dr = LLE(n_components = 2, n_neighbors = n_neighbors)
        
    elif dr_name=='SE':
        dr = SE(n_components = 2, n_neighbors = n_neighbors)
        
    elif dr_name=='UMAP':
        dr = umap.UMAP(n_components = 2, n_neighbors = n_neighbors, verbose=False, min_dist=min_dist)
        
    elif dr_name=='GRP':
        dr = GRP(n_components = 2)
        
    dr_data = dr.fit_transform(features)
    dr_data = pd.DataFrame(dr_data, columns=['{}_1'.format(dr_name),'{}_2'.format(dr_name)])
    ###########
    
    ## save stuff
    if labels is not None:
        dr_data['labels'] = list(labels)
        fig, ax = plt.subplots()
        sns.scatterplot(dr_data['{}_1'.format(dr_name)], dr_data['{}_2'.format(dr_name)], hue = dr_data['labels'] )
        plt.savefig(dataset_name + '/' + figurefolder+ '/' + file_name +'.pdf')
    
    
    dr_data.to_csv(dataset_name + '/' + datafolder +'/' +file_name + '.csv', index=False)
    print('---------Finished: {}-----------'.format(file_name))
    ##########
    
    return 
    

            

def load_and_combine(folder):
    files = os.listdir(folder)
    datasets = []
    for f in files:
        datasets.append(pd.read_csv(folder+files))
    
    return pd.concat(datasets, axis=1)


In [19]:
# apply dr

files = os.listdir()
files = ['Caltech_100_instruments']
print(files)

def apply_results(files,csv, datafolder, figurefolder):
    for f in files:
        if os.path.isdir(f) and not f.startswith('.') and not f.startswith('_') and not f.startswith('MNIST'):
            print(f)
            for dr_name in ['PCA','TSNE', 'ISM', 'SE', 'GRP', 'UMAP', 'LLE']:
                print(dr_name)
                if dr_name in ['PCA','GRP']:
                    dr_data = apply_dr(datafolder, figurefolder,dr_name, f+'/'+ csv, dataset_name = f)
                elif dr_name == 'TSNE':
                    for p in [5,10,30,50,150,100]:
                        #print(p)
                        dr_data = apply_dr(datafolder, figurefolder,dr_name,f+'/'+ csv, perplexity=p, dataset_name = f)   
                else:
                    for n in [2,3,5,7,10,15]:
                        #print(n)
                        if dr_name == 'UMAP':
                            for d in [0.05, 0.1, 0.25, 0.5, 0.8, 0.99]:
                                dr_data = apply_dr(datafolder, figurefolder,dr_name,f+'/'+ csv, n_neighbors=n, min_dist=d, dataset_name = f)
                        else:
                            dr_data = apply_dr(datafolder, figurefolder,dr_name, f+'/'+ csv, n_neighbors=n, dataset_name = f)

#apply_results(files, 'flat_img_50.csv', 'data', 'figures')                   


['Caltech_100_instruments']


In [72]:
def getImage(path):
    return OffsetImage(plt.imread(path))


#@ray.remote
def plot_projection(f, cp, csv, labels, paths,idx, figurefolder, datafolder):
    print(csv)
    orig_path = f + '/' + datafolder +'/'
    try:
        df = pd.read_csv(orig_path + csv)
    except: 
        return

    df = df.loc[idx,:]
    #print(df)
    df['labels'] = labels

    fig, ax = plt.subplots(figsize=(100,100))
    ax.scatter(df.iloc[:,0], df.iloc[:,1]) 
    for i,row in df.iterrows():
        #try:
        if paths[i].startswith('.'):
            continue
        print(row)
        img = f+'/'+'thumbnails'+'/' + paths[i]
        ab = AnnotationBbox(getImage(img), (row[0], row[1]), frameon=True)
#         except:
#             try:
#                 ab = AnnotationBbox(getImage(row['paths'].split('.png')[0]+'.jpg'), (row[0], row[1]), frameon=True)
#             except:
#                 continue
        ab.patch.set_linewidth(4)
        ab.patch.set_edgecolor(cp[int(row['labels']) if row['labels']<11 else 11])
        ab.patch.set_facecolor(cp[int(row['labels']) if row['labels']<11 else 11])
        ax.add_artist(ab)
        

    print(f+'/' + figurefolder+ '/img_'+csv.split('.csv')[0]+'.pdf')
    plt.savefig(f+'/' + figurefolder +'/img_'+csv.split('.csv')[0]+'.pdf')
    plt.close()

 

           

In [75]:
files = ['stanford']
def apply_projection(path, datafolder, thumbnailsfolder, figurefolder, files): 
    cp = sns.color_palette("Paired")

    for f in files:
        orig_path = f + '/' + datafolder +'/'
        print(orig_path)
        try:
            ddf = pd.read_csv(f + '/' + datafolder+'/PCA.csv')
        except:
            continue

        if len(ddf)>400:
            _, ddf = train_test_split(ddf, test_size=400, stratify=ddf['labels'])

        idx = ddf.index
        print(ddf)
        paths = list(os.listdir('stf/thumbnails/'))
        #paths = list(f+'/' + thumbnailsfolder+'/'+ ddf['filename']+'.jpg')
        #paths = list(f+'/' + thumbnailsfolder+'/'+ ddf['filename']+'.gif')
        
        le = preprocessing.LabelEncoder()
        labels = le.fit_transform(ddf['labels'])

        for csv in os.listdir(orig_path):
            if csv.endswith('.csv'):
                if os.path.exists(f+'/'+figurefolder+'/img_'+csv.split('.csv')[0]+'.pdf'):
                    print('.')
                    continue 
                plot_projection(f, cp, csv, labels, paths,idx, figurefolder,datafolder)


#apply_projection('stanford', 'data', 'thumbnails', 'figures',files)